In [1]:
from langchain_community.document_loaders import DirectoryLoader
import streamlit as st




In [ ]:
loader = DirectoryLoader("resources/data/engineering")
docs = loader.load()
print(len(docs))

FileNotFoundError: Directory not found: 'resources/data/{role}'

In [4]:
print(docs[0].page_content[:1000])

FinSolve Technologies Engineering Document

1. Introduction

1.1 Company Overview

FinSolve Technologies is a leading FinTech company headquartered in Bangalore, India, with operations across North America, Europe, and Asia-Pacific. Founded in 2018, FinSolve provides innovative financial solutions, including digital banking, payment processing, wealth management, and enterprise financial analytics, serving over 2 million individual users and 10,000 businesses globally.

1.2 Purpose

This engineering document outlines the technical architecture, development processes, and operational guidelines for FinSolve's product ecosystem. It serves as a comprehensive guide for engineering teams, stakeholders, and partners to ensure alignment with FinSolve's mission: "To empower financial freedom through secure, scalable, and innovative technology solutions."

1.3 Scope

This document covers:

System architecture and infrastructure

Software development lifecycle (SDLC)

Technology stack

Security 

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
documents = text_splitter.split_documents(docs)
documents[:5]

[Document(metadata={'source': 'resources\\data\\engineering\\engineering_master_doc.md'}, page_content='FinSolve Technologies Engineering Document\n\n1. Introduction\n\n1.1 Company Overview\n\nFinSolve Technologies is a leading FinTech company headquartered in Bangalore, India, with operations across North America, Europe, and Asia-Pacific. Founded in 2018, FinSolve provides innovative financial solutions, including digital banking, payment processing, wealth management, and enterprise financial analytics, serving over 2 million individual users and 10,000 businesses globally.\n\n1.2 Purpose\n\nThis engineering document outlines the technical architecture, development processes, and operational guidelines for FinSolve\'s product ecosystem. It serves as a comprehensive guide for engineering teams, stakeholders, and partners to ensure alignment with FinSolve\'s mission: "To empower financial freedom through secure, scalable, and innovative technology solutions."\n\n1.3 Scope\n\nThis docu

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents[:],OllamaEmbeddings())

C:\Users\nalla\AppData\Local\Temp\ipykernel_25908\2625238470.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  db = Chroma.from_documents(documents[:],OllamaEmbeddings())


In [7]:
query = "what's the scope"
result = db.similarity_search(query)
result[0].page_content

'Post-incident analysis and lessons learned\n\n5.3.3 Security Training\n\nMandatory security awareness training for all employees\n\nRole-specific security training for developers, administrators\n\nQuarterly phishing simulations\n\nSecurity champions program within engineering teams\n\n6. Testing and Quality Assurance\n\n6.1 Testing Strategy\n\n6.1.1 Test Pyramid\n\nUnit Tests:\n\nCover 90% of code base\n\nFocus on business logic and edge cases\n\nImplemented using Jest (Node.js), Pytest (Python), Go testing\n\nIntegration Tests:\n\nValidate microservice interactions\n\nTest database operations and external service integrations\n\nImplemented using Postman/Newman and custom test harnesses\n\nEnd-to-End Tests:\n\nSimulate complete user journeys\n\nCover critical business flows\n\nImplemented with Cypress (web) and Appium (mobile)\n\n6.1.2 Specialized Testing\n\nPerformance Testing:\n\nLoad testing with JMeter (target: 2,000 concurrent users)\n\nStress testing to identify breaking point

In [8]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama2")
llm

OllamaLLM(model='llama2')

In [28]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    
    """
    You are a helpful assistant who helps users to search all <context> you have and provide answer to them and addtionally provide them the link to that file where you have fetched the answer from.
    <context>
    {context}
    </context>

    input: {input}
    """
)

In [29]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm,prompt)

In [30]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001EC892B1630>, search_kwargs={})

In [31]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [33]:
response = retrieval_chain.invoke({"input": "I am fresher just joined what techstack should I use?"})
print(response['answer'])

Hello there! As a helpful assistant, I'm glad you asked me for advice on choosing the right tech stack for your project. Based on the context you provided, here are some recommendations for a suitable tech stack:

1. Programming Languages:
	* Go Lang: A popular choice for building microservices architectures, especially in the financial technology (FinTech) industry. It offers a perfect blend of simplicity, reliability, and performance.
	* Java/Kotlin: A widely used language in the enterprise sector, particularly for building robust backend systems. These languages are known for their platform independence, which allows you to deploy your application across various environments.
2. Frameworks and Libraries:
	* Spring Boot: An excellent choice for building microservices architectures, especially when combined with Go Lang or Java/Kotlin. It provides a comprehensive set of tools for developing, testing, and deploying applications.
	* Reactor: A lightweight framework that simplifies async